# Setup

In [1]:
from IPython.display import display, Markdown, Math, HTML, Code
from mogwai.parser import graphml_to_mogwaigraph
from mogwai.core.traversal import MogwaiGraphTraversalSource
from mogwai.core.steps.statics import *
import os

In [2]:
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.traversal import P, T
from gremlin_python.process.graph_traversal import __

In [3]:
# SERVER STARTEN
import subprocess
gremlin_server_path = '/home/joep/programs/gremlin-server'
gremlin_command = f'{gremlin_server_path}/bin/gremlin-server.sh {gremlin_server_path}/conf/gremlin-server.yaml'
gremlin_server_process = subprocess.Popen(gremlin_command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
display(Markdown("#### ✔ Server gestartet."))

#### ✔ Server gestartet.

In [60]:
gremlin_server_process.kill()

In [4]:
# MIT SERVER VERBINDEN
# Connect to localhost:8182
graph_gp = Graph()
g_gp = graph_gp.traversal().with_remote(DriverRemoteConnection("ws://127.0.0.1:8182/gremlin", "g", transport_factory=lambda:AiohttpTransport(call_from_event_loop=True)))
display(Markdown("#### ✔ Mit Server Verbunden."))

#### ✔ Mit Server Verbunden.

# Large Airroutes Graph

## Import Graph

In [5]:
graph_mogwai = graphml_to_mogwaigraph("tests/documents/air-routes-latest.graphml", node_label_key="labelV", edge_label_key="labelE", node_name_key="code")
g_m = MogwaiGraphTraversalSource(graph_mogwai)

In [6]:
air_routes_path=os.path.abspath("tests/documents/air-routes-latest.graphml")
# read the content from the air routes example
g_gp.V().drop().iterate()
g_gp.io(air_routes_path).read().iterate()
v_count=g_gp.V().count().next()
print (f"graph imported from {air_routes_path} has {v_count} vertices")

graph imported from /home/joep/OneDrive/Documents/SS 2024/KG Lab/pymogwai/tests/documents/air-routes-latest.graphml has 3749 vertices


## Query 1

### PyMogwai

In [11]:
g_m.V().has_label('airport').has_name('LAX').as_('start').repeat(
		outE('route').as_('e').inV().filter_(
			select('e').properties('dist').is_(lte(2000))
		).simple_path()
	).times(3).emit().properties("city").dedup().to_list().run()

['Atlanta',
 'Austin',
 'Nashville',
 'Dallas',
 'Houston',
 'Minneapolis',
 'Chicago',
 'Phoenix',
 'Seattle',
 'San Francisco',
 'San Jose',
 'San Diego',
 'Salt Lake City',
 'Las Vegas',
 'Denver',
 'San Antonio',
 'New Orleans',
 'Cedar Rapids',
 'El Paso',
 'Oakland',
 'Tucson',
 'Santa Fe',
 'Detroit',
 'Vancouver',
 'Edmonton',
 'Calgary',
 'Mexico City',
 'Portland',
 'Oaklahoma City',
 'Palm Springs',
 'Memphis',
 'Cincinnati',
 'Indianapolis',
 'Kansas City',
 'St Louis',
 'Albuquerque',
 'Milwaukee',
 'Omaha',
 'Tulsa',
 'Puerto Vallarta',
 'Sacramento',
 'Colorado Springs',
 'Louisville',
 'Boise',
 'Reno',
 'Columbus',
 'Wichita',
 'Springfield',
 'Aguascalientes',
 'Monterrey',
 'Silao',
 'Durango',
 'Fresno',
 'Guadalajara',
 'Grand Junction',
 'Morelia',
 'Madison',
 'Mazatlán',
 'San José del Cabo',
 'Fayetteville/Springdale/',
 'Zacatecas',
 'Eagle',
 'Hayden',
 'South Bend',
 'Arcata/Eureka',
 'Bakersfield',
 'Eugene',
 'Medford',
 'Monterey',
 'Pasco',
 'Redmond',
 

### Gremlin-Python

In [12]:
g_gp.V().has_label('airport').has("code",'LAX').as_('start').repeat(
        __.outE('route').as_('e').inV().filter_(
            __.select('e').values('dist').is_(P.lte(2000))
        ).simple_path()
    ).times(3).emit().values('city').dedup().to_list()

['Houston',
 'Fayetteville/Springdale/',
 'San José del Cabo',
 'Eagle',
 'Minneapolis',
 'Zacatecas',
 'Chicago',
 'Hayden',
 'Dallas',
 'Mazatlán',
 'Madison',
 'Morelia',
 'San Francisco',
 'San Jose',
 'San Diego',
 'Vancouver',
 'Salt Lake City',
 'Las Vegas',
 'Denver',
 'Phoenix',
 'Seattle',
 'El Paso',
 'Oakland',
 'Tucson',
 'San Antonio',
 'New Orleans',
 'Cedar Rapids',
 'South Bend',
 'Santa Fe',
 'Aspen',
 'Kelowna',
 'San Luis Obispo',
 'Bakersfield',
 'Arcata/Eureka',
 'Medford',
 'Eugene',
 'Pasco',
 'Monterey',
 'Santa Barbara',
 'Redmond',
 'Santa Maria',
 'Visalia',
 'Edmonton',
 'Santa Rosa',
 'Detroit',
 'Carlsbad',
 'Bellingham',
 'Kingman',
 'Spokane',
 'Mammoth Lakes',
 'Merced',
 'Prescott',
 'Yuma',
 'Bozeman',
 'Provo',
 'Calgary',
 'Hermosillo',
 'Durango',
 'Uruapan',
 'Loreto',
 'Manzanillo',
 'Ixtapa',
 'Oaxaca',
 'Oaklahoma City',
 'Imperial',
 'Portland',
 'Flagstaff',
 'Hailey',
 'Ogden',
 'Jackson',
 'Kalispell',
 'St George',
 'Mexico City',
 'Misso

## Query 2

In [ ]:
g_m.V().has_label("airport").within("city", ["Brussels", "Maastricht", "Aachen", "Dusseldorf"]).as_('start').\
        outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest').order(desc=True).by(select('e').properties('dist')).\
        limit(5).select('start', 'dest').to_list().by('desc').run()

## Pymogwai

In [9]:
from mogwai.core.steps.statics import select
query = g_m.V().has_label("airport").within(["properties","city"], ["Brussels", "Maastricht", "Aachen", "Dusseldorf"]).as_('start')\
        .outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest')\
        .order(desc=True).by(select('e').properties('dist')).\
        limit(5).select('start', 'dest').by('desc').to_list()
res = query.run()
print("Result:", res)

Result: [['Dusseldorf International Airport', 'Singapore'], ['Brussels Airport', 'Japan'], ['Brussels South Charleroi Airport', 'Hong Kong'], ['Brussels Airport', 'Hong Kong'], ['Brussels Airport', 'China']]


## Gremlin-Python

In [10]:
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import P
query = g_gp.V().has_label("airport").has("city", P.within(["Brussels", "Maastricht", "Aachen", "Dusseldorf"])).as_('start')\
            .outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest')\
            .order().by(__.select('e').values('dist'),Order.desc)\
            .limit(5).select('start', 'dest').by("desc").to_list()
print(query)

[{'start': 'Dusseldorf International Airport', 'dest': 'Singapore'}, {'start': 'Brussels Airport', 'dest': 'Japan'}, {'start': 'Brussels South Charleroi Airport', 'dest': 'Hong Kong'}, {'start': 'Brussels Airport', 'dest': 'Hong Kong'}, {'start': 'Brussels Airport', 'dest': 'China'}]


# Query 3

## PyMogwai

In [43]:

from mogwai.core.steps.statics import outE, select, lte, count, Scope
query = g_m.V().has_label('airport').has_property("city", "Maastricht").as_('start').repeat(
            outE('route').as_('e').inV().filter_(
                select('e').properties('dist').is_(lte(2000))
            ).simple_path()
        ).times(3).emit().as_('dest')\
        .path().as_('p').count(Scope.local).order(asc=True).as_('length')\
        .select('dest').by('city').dedup().select('dest', 'length').to_list().by('city')
print("Query:", query.print_query())
res = query.run()
print(res)
#print("Result length", len(res))

Query: V -> Contains -> Has -> As -> Repeat(OutE -> As -> InV -> Filter(Select -> Value -> Is) -> SimplePath, x3) -> As -> Path -> As -> Count -> Order -> As -> Select -> Dedup -> Select -> ToList
[['Alicante', 3], ['Alghero', 3], ['Trapani', 3], ['Girona', 3], ['Catania', 3], ['Kerkyra Island', 3], ['Heraklion', 3], ['Tenerife Island', 3], ['Bari', 3], ['Porto', 3], ['Faro', 3], ['Zakynthos', 3], ['Antalya', 3], ['Palma De Mallorca', 3], ['Istanbul', 3], ['Luqa', 3], ['Kos Island', 3], ['Geneva', 5], ['Liverpool', 5], ['Nottingham', 5], ['London', 5], ['Shannon', 5], ['Manchester', 5], ['Brussels', 5], ['Zurich', 5], ['Kalymnos Island', 5], ['Barcelona', 5], ['Rome', 5], ['Dublin', 5], ['Paris', 5], ['Berlin', 5], ['Madrid', 5], ['Nice', 5], ['Luxembourg', 5], ['Lyon', 5], ['Cork', 5], ['Basle', 5], ['Sevilla', 5], ['Bristol', 5], ['Leeds', 5], ['Malaga', 5], ['Milan', 5], ['Gran Canaria', 5], ['Lisbon', 5], ['Doncaster', 5], ['Nantes', 5], ['Bologna', 5], ['Pisa', 5], ['Naples', 5], 

## Gremlin-Python

In [41]:
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Scope
query = g_gp.V().has_label('airport').has("city", "Maastricht").as_('start').repeat(
				__.outE('route').as_('e').inV().filter_(__.select('e').values('dist').is_(P.lte(2000))
			).simple_path()
        ).times(3).emit().as_('dest')\
        .path().as_('p').count(Scope.local).order().by(Order.asc).as_('length')\
        .select("dest").by("city").dedup().as_("dest").select('dest', 'length').to_list()
print(query)

[{'dest': 'Porto', 'length': 3}, {'dest': 'Faro', 'length': 3}, {'dest': 'Zakynthos', 'length': 3}, {'dest': 'Alghero', 'length': 3}, {'dest': 'Antalya', 'length': 3}, {'dest': 'Trapani', 'length': 3}, {'dest': 'Palma De Mallorca', 'length': 3}, {'dest': 'Girona', 'length': 3}, {'dest': 'Catania', 'length': 3}, {'dest': 'Istanbul', 'length': 3}, {'dest': 'Luqa', 'length': 3}, {'dest': 'Bari', 'length': 3}, {'dest': 'Alicante', 'length': 3}, {'dest': 'Kerkyra Island', 'length': 3}, {'dest': 'Heraklion', 'length': 3}, {'dest': 'Tenerife Island', 'length': 3}, {'dest': 'Kos Island', 'length': 3}, {'dest': 'Beauvais/Tillé', 'length': 5}, {'dest': 'Warsaw', 'length': 5}, {'dest': 'London', 'length': 5}, {'dest': 'Naples', 'length': 5}, {'dest': 'Roma', 'length': 5}, {'dest': 'Bergamo', 'length': 5}, {'dest': 'Brussels', 'length': 5}, {'dest': 'Katowice', 'length': 5}, {'dest': 'Lisbon', 'length': 5}, {'dest': 'Luxembourg', 'length': 5}, {'dest': 'Gran Canaria', 'length': 5}, {'dest': 'Venic

In [42]:
len(query)

880

## Mogwai

In [59]:
g_m.V().has_label("country").has_property("desc", "Germany").out("contains").outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest').select('e').order(desc=True).by("dist").limit(1).select('dest').by('desc').next().run()

'Argentina'

## Gremlin Python

In [45]:
g_gp.V().has_label("country").has("desc", "Germany").out("contains").outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest').select('e').order().by("dist",Order.desc).limit(1).select("dest").by("desc").next()

'Argentina'

# execute queries mutliple times

In [ ]:
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Scope
import time
for i in range(10):
    query = g_gp.V().has_label('airport')\
        .has("city", "Maastricht").as_('start')\
            .repeat(
                __.outE('route').as_('e').inV().filter_(__.select('e').values('dist').is_(P.lte(2000))).simple_path()
            )\
                .times(3).emit().as_('dest').path().as_('p')\
                    .count(Scope.local).order().by(Order.asc)\
                        .as_('length').select("dest").by("city").dedup().as_("dest").select('dest', 'length').to_list()

In [ ]:
from mogwai.core.steps.statics import outE, select, lte, count, Scope
for i in range(10):
    query = g_m.V().has_label('airport').has_property("city", "Maastricht").as_('start').repeat(
                outE('route').as_('e').inV().filter_(select('e').properties('dist').is_(lte(2000))).simple_path()
            ).times(3).emit().as_('dest').path().as_('p').count(Scope.local).order(asc=True).as_('length').select('dest').dedup().select('dest', 'length').to_list().by('city')
    print("Query:", query.print_query())
    res = query.run()